1. Prove formula (6.7) in the textbook, i.e. show that the variance of a sum of random but correlated variables can be written as $${\rm Var}\left(\frac{1}{m}\sum_{i=1}^m x_i\right)= \rho \sigma^2 + \frac{1}{m}(1-\rho)\sigma^2,$$
where ${\rm Var}(x_i)=\sigma^2$ and the correlation coefficient $\rho_{x_i,x_j}={\rm Cov(x_i,x_j)}/\sigma^2$

(Hint: Review some properties of the covariance)

You can write out the calculation by hand and attach a scanned pdf.

(https://en.wikipedia.org/wiki/Variance#Linear_combinations):

$${\rm Var}\left(\frac{1}{m}\sum_{i=1}^m x_i\right) = \sum_{i=1}^m {\rm Var}\left(\frac{x_i}{m}\right) + 2 \sum_{i,j=1; i<j}^m {\rm Cov}\left(\frac{x_i}{m}, \frac{x_j}{m} \right)$$

$$= \sum_{i=1}^m \frac{\sigma^2}{m^2} + 2\sum_{i,j=1; i<j}^m \frac{\rho\sigma^2}{m^2}$$

$$= m \frac{\sigma^2}{m^2} + 2 \left(\frac{m(m-1)}{2} \frac{\rho\sigma^2}{m^2}\right)$$

$$= \frac{\sigma^2}{m} + (m-1) \frac{\rho\sigma^2}{m}$$

$$= \frac{\sigma^2}{m} + \rho\sigma^2 - \frac{\rho\sigma^2}{m}$$

$$= \rho\sigma^2 + \frac{\sigma^2}{m}  - \frac{\rho\sigma^2}{m}$$

$$= \rho \sigma^2 + \frac{1}{m}(1-\rho)\sigma^2$$

2. Load again the cleaned dataset from Lab 13 for the photometric redshift prediction with 6,307 objects and 6 features (sel_feature.csv and sel_target.csv). You can also just re-do the data cuts from the original file if you prefer.

Optimize (using a Grid Search for the parameters you deem to be most relevant) the 	Extremely Random Tree algorithm and compute performance metric and the outlier fraction. How do they compare to the optimal Random Forest model? Comment not just on the 	scoring parameter(s), but also on high variance vs high bias. Which model would you pick?

**- comparing the two models below. We achieve perfect train scores for Extremely Random Tree and near perfect for random forest.**
**- We see a difference in choice for criterion (loss function) -> the best choice for random forest is absolute error while extremely random is squared error.** 
**- outlier fraction is zero for extremely random and close to zero for random forest.**
**- the best test score is also slightly larger for extremely random (76% compared to 73%)**

**Based on the facts given above, i would recommend Extremely Random Tree. But in both cases we can see an issue of high variance as our train scores are (near) perfect in either case but our test scores hover around ~75%**

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
# read in pre-processed data from Lab 13
sel_features = pd.read_csv('sel_features.csv', sep = '\t')
sel_target = pd.read_csv('sel_target.csv')
sel_features = sel_features.to_numpy()
sel_target = sel_target.to_numpy().T[0]

In [3]:
model = RandomForestRegressor()

parameters = {'max_depth':[6,10,None], 'criterion':['squared_error','absolute_error'],
              'n_estimators':[20,50,100]}

gmodel = GridSearchCV(model, parameters, cv = KFold(n_splits=3, shuffle=True), \
                     verbose = 0, n_jobs = -1, return_train_score = True)

gmodel.fit(sel_features, sel_target)

test_scores = gmodel.cv_results_['mean_test_score']
train_scores = gmodel.cv_results_['mean_train_score']
print('Train Scores {}:'.format(train_scores))
print('')

print('Test Scores {}:'.format(test_scores))
print('')
print('Best params, best score:', "{:.4f}".format(gmodel.best_score_), gmodel.best_params_)

ypred = gmodel.predict(sel_features)
outlier_frac = len(sel_target[np.abs(sel_target - ypred)/(1 + ypred) > 0.15])/len(sel_target)
print('')
print('outlier fraction: {}'.format(outlier_frac))

Train Scores [0.73759005 0.7328299  0.73440394 0.91912315 0.9251829  0.92719805
 0.95942045 0.96166596 0.96336956 0.70622155 0.71243057 0.71483655
 0.91284342 0.91769304 0.92022102 0.95607143 0.96186797 0.9625043 ]:

Test Scores [0.60625093 0.60317042 0.60421722 0.69668757 0.71391262 0.71189669
 0.71951199 0.7301097  0.73296534 0.6112628  0.61291798 0.61787345
 0.71060109 0.71145727 0.7178372  0.72042418 0.72960046 0.73351517]:

Best params, best score: 0.7335 {'criterion': 'absolute_error', 'max_depth': None, 'n_estimators': 100}

outlier fraction: 0.008244807356905026


In [4]:
model = ExtraTreesRegressor()

parameters = {'max_depth':[6,10,None], 'criterion':['squared_error','absolute_error'],
              'n_estimators':[20,50,100]}

gmodel = GridSearchCV(model, parameters, cv = KFold(n_splits=3, shuffle=True), \
                     verbose = 0, n_jobs = -1, return_train_score = True)

gmodel.fit(sel_features, sel_target)

test_scores = gmodel.cv_results_['mean_test_score']
train_scores = gmodel.cv_results_['mean_train_score']
print('Train Scores {}:'.format(train_scores))
print('')
print('Test Scores {}:'.format(test_scores))
print('')
print('Best params, best score:', "{:.4f}".format(gmodel.best_score_), gmodel.best_params_)

ypred = gmodel.predict(sel_features)
outlier_frac = len(sel_target[np.abs(sel_target - ypred)/(1 + ypred) > 0.15])/len(sel_target)
print('')
print('outlier fraction: {}'.format(outlier_frac))

Train Scores [0.49880956 0.50804082 0.51872758 0.85209996 0.85879361 0.85574545
 1.         1.         1.         0.49633524 0.50135561 0.49420418
 0.83289862 0.84167807 0.83604222 1.         1.         1.        ]:

Test Scores [0.43775095 0.44923564 0.4575736  0.6799281  0.68413239 0.68509405
 0.738195   0.75928228 0.76359834 0.45633272 0.45730127 0.45268354
 0.68466203 0.68961454 0.68601803 0.7456567  0.757229   0.75593366]:

Best params, best score: 0.7636 {'criterion': 'squared_error', 'max_depth': None, 'n_estimators': 100}

outlier fraction: 0.0
